In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
class KNN:#以下所有函数除标签外均接收二维数组
    def __init__(self,traindata,label,k=3,p=2):
        self.label=None
        self.K=k#传入K值
        self.p=p#传入p值，默认为2
        self.X=traindata
        self.label=label
    
    def L(self,X1,X2):#计算两个样本特征值的距离,为避免误判，将两个样本设为X1,X2
        if len(X1)==len(X2) and len(X1)>1:
            distance=0
            for i in range(len(X1)):
                distance+=math.pow(abs(X2[i]-X1[i]),self.p)#按照闵可夫斯距离计算
            return math.pow(distance,1/self.p)
        else:
            return 0
        
    def fit(self,y):#计算每一个样本与测试的结果，并返回一个长度为K的列表，列表元素为值和标签组成的元组
        numtrainx=len(self.X)#由于一般的训练集都不会只有一个样本，因此，不考虑只有一个样本的情况
        lenlist=self.K
        result1=[]
        for i in range(lenlist):#首先挑选前K各样本，计算前K各样本与y的距离，并填入一个列表中
            distance=self.L(self.X[i],y)
            result1.append((self.label[i],distance))
        for num1 in range(lenlist,numtrainx):#计算剩下的样本与y的距离
            distance1=self.L(self.X[num1],y)
            indexmin=result1.index(max(result1,key=lambda x:x[1]))#此项为得到一个列表中最大值所对应的索引，后续每个样本与y的距离只需要更新result这个列表即可
            if distance1<max(result1,key=lambda x:x[1])[1]:#更新条件是每各样本计算出来距离后，就跟列表中的最大值比较，比最大值小，就更新，否则不更新
                result1[indexmin]=(self.label[num1],distance1)
        return result1    
    
    def predict(self,testdata):
        predcitlabel=[]
        for i in range(len(testdata)):
            resultdistance=self.fit(testdata[i])
            label=[i[0] for i in resultdistance]#获取对应距离的标签，并放入组成一个列表
            labelcount={}#计算列表中每一个标签出现的次数，将结果放入字典中
            for num1 in label:
                labelcount1=label.count(num1)
                labelcount[num1]=labelcount1
            maxlabel=max(labelcount,key=labelcount.get)
            predcitlabel.append(maxlabel)
        return predcitlabel
    
    def score(self,testdata,testlabel):
        rightcount=0
        for p1,y1 in zip(testdata,testlabel):
            p2=np.array([p1])#因为定义的函数需要传入二维数组
            label=self.predict(p2)
            if label==y1:
                rightcount+=1
        return rightcount/len(testdata)
                

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter

iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']

data = np.array(df.iloc[:100, [0, 1, -1]])
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf=KNN(X_train,y_train)
clf.score(X_test,y_test)

1.0